#Data Manipulation Project

---

In [1]:
import pandas as pd
import numpy as np

In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

In [39]:
#We import two dataframes with the answers and the questions
path='//content/drive/MyDrive/DataSets'
df=pd.read_csv(path+'/Raw_Data_Survey_project.csv',sep=",",encoding='latin-1') 
questions=pd.read_csv(path+'/question_dataset_survey_project.csv', sep=',')

In [40]:
# Part of a good practice is to create a copy of the data, just in case there is mistake, you can always go back to the original dataset
df_modified=df.copy()
df_modified.tail()

,Respondent ID,Start Date,End Date,Identify which division you work in.-Response,Identify which division you work in.-Other (please specify),Which of the following best describes your position level?-Response,Which generation are you apart of?-Response,Please select the gender in which you identify.-Response,Which duration range best aligns with your tenure at your company?-Response,Which of the following best describes your employment type?-Response,Question 1-Response,Question 2-Response,Question 3-Open-Ended Response,Question 4-Response,Question 4-Other (please specify),Question 5-Response 1,Question 5-Response 2,Question 5-Response 3,Question 5-Response 4,Question 5-Response 5,Question 5-Response 6,Question 6-Response 1,Question 6-Response 2,Question 6-Response 3,Question 6-Response 4,Question 6-Response 5,Question 6-Response 6,Question 7-Response 1,Question 7-Unscheduled,Question 8-Response 1,Question 8-Response 2,Question 8-Response 3,Question 8-Response 4,Question 9-Response 1,Question 9-Response 2,Question 9-Response 3,Question 9-Response 4,Question 10-Response 1,Question 10-Response 2,Question 10-Response 3,...,Question 22-Reponse 2,Question 23-Response,Question 24-Response 1,Question 24-Response 2,Question 24-Response 3,Question 24-Response 4,Question 24-Response 5,Question 25-Response 1,Question 25-Response 2,Question 25-Response 3,Question 25-Response 4,Question 25-Response 5,Question 25-Response 6,Question 25-Response 7,Question 25-Response 8,Question 25-Response 9,Question 26-Response 1,Question 26-Response 2,Question 26-Response 3,Question 26-Response 4,Question 27-Response 1,Question 27-Response 2,Question 28-Response,Question 29-Response 1,Question 29-Response 2,Question 29-Response 3,Question 29-Response 4,Question 29-Response 5,Question 29-Response 6,Question 29-Response 7,Question 29-Response 8,Question 29-Response 9,Question 29-Response 10,Question 29-Response 11,Question 29-Response 12,Question 29-Response 13,Question 29-Response 14,Question 30-Response 1,Question 30-Response 2,Question 30-Response 3
193,7940065082,2021-01-11 06:19:18,2021-01-11 06:32:59,Infrastructure,NaN,Department Lead,Baby Boomer (born between 1946-1964),Male,10+ years,Full time Employee,Answer 3,NaN,Answer 2,Answer 4,Answer 5,Answer 7,NaN,Answer 5,NaN,NaN,NaN,NaN,Answer 5,Answer 3,NaN,NaN,NaN,NaN,Answer 2,NaN,Answer 3,NaN,NaN,NaN,Answer 6,Answer 7,Answer 6,Answer 7,NaN,NaN,...,NaN,NaN,Answer 7,Answer 5,NaN,Answer 5,NaN,Answer 4,Answer 7,Answer 4,NaN,Answer 4,NaN,NaN,NaN,Answer 1,Answer 8,NaN,Answer 1,NaN,NaN,Answer 5,NaN,Answer 3,NaN,Answer 8,NaN,NaN,Answer 1,Answer 2,Answer 1,Answer 1,Answer 3,Answer 6,Answer 6,Answer 8,NaN,NaN,NaN,Answer 8
194,5157705612,2021-01-11 06:19:14,2021-01-11 06:33:29,Finance,NaN,Staff,Millennial (born between 1981-2000),Female,5-10 years,Full time Employee,NaN,NaN,Answer 7,NaN,Answer 2,Answer 8,Answer 1,Answer 2,NaN,Answer 6,Answer 3,NaN,NaN,NaN,NaN,Answer 4,NaN,Answer 1,Answer 6,Answer 6,Answer 8,Answer 1,Answer 4,Answer 6,Answer 2,Answer 2,NaN,Answer 5,Answer 4,Answer 4,...,Answer 1,Answer 1,Answer 3,NaN,Answer 1,Answer 5,Answer 6,Answer 2,Answer 6,NaN,Answer 5,Answer 3,Answer 7,Answer 3,NaN,Answer 3,Answer 4,Answer 6,NaN,NaN,Answer 1,Answer 8,Answer 7,Answer 2,NaN,Answer 6,Answer 3,Answer 7,Answer 4,Answer 5,Answer 5,NaN,Answer 5,NaN,NaN,Answer 2,NaN,NaN,NaN,Answer 6
195,9920755555,2021-01-11 06:18:48,2021-01-11 06:27:27,Port Operations,NaN,Staff,Millennial (born between 1981-2000),Female,3-5 years,Full time Employee,NaN,Answer 7,Answer 2,NaN,Answer 3,Answer 5,Answer 7,NaN,Answer 3,NaN,Answer 4,Answer 5,NaN,NaN,NaN,Answer 3,NaN,NaN,Answer 4,Answer 7,Answer 6,NaN,NaN,NaN,NaN,Answer 8,Answer 6,NaN,NaN,Answer 6,...,Answer 2,NaN,NaN,NaN,Answer 4,NaN,Answer 1,Answer 4,NaN,Answer 2,Answer 6,NaN,Answer 7,NaN,NaN,Answer 3,Answer 3,NaN,Answer 7,Answer 4,NaN,NaN,Answer 6,Answer 2,Answer 5,NaN,Answer 5,Answer 4,Answer 5,NaN,NaN,Answer 7,NaN,NaN,NaN,NaN,NaN,NaN,Answer 3,NaN
196,6638341389,2021-01-11 06:19:01,2021-

In [41]:
#I Identify 2 columns we don't need, I drop them inplace
df_modified.drop(['Start Date', 'End Date'],axis=1,inplace=True) 


In [42]:
#I identified the demographic variables and the actual values and we turn them into lists
id_vars = list(df_modified.columns)[:8]
value_vars= list(df_modified.columns)[8:]
print('id_vars:',id_vars)
print ('value vars:',value_vars)

id_vars: ['Respondent ID', 'Identify which division you work in.-Response', 'Identify which division you work in.-Other (please specify)', 'Which of the following best describes your position level?-Response', 'Which generation are you apart of?-Response', 'Please select the gender in which you identify.-Response', 'Which duration range best aligns with your tenure at your company?-Response', 'Which of the following best describes your employment type?-Response']
value vars: ['Question 1-Response', 'Question 2-Response', 'Question 3-Open-Ended Response', 'Question 4-Response', 'Question 4-Other (please specify)', 'Question 5-Response 1', 'Question 5-Response 2', 'Question 5-Response 3', 'Question 5-Response 4', 'Question 5-Response 5', 'Question 5-Response 6', 'Question 6-Response 1', 'Question 6-Response 2', 'Question 6-Response 3', 'Question 6-Response 4', 'Question 6-Response 5', 'Question 6-Response 6', 'Question 7-Response 1', 'Question 7-Unscheduled', 'Question 8-Response 1', 'Qu

In [43]:
#Let´s apply the melt method to melt the dataframe
df_melted=df_modified.melt(id_vars=id_vars,value_vars=value_vars,var_name='question_&_subquestion',value_name='Answer')
df_melted.head() 

,Respondent ID,Identify which division you work in.-Response,Identify which division you work in.-Other (please specify),Which of the following best describes your position level?-Response,Which generation are you apart of?-Response,Please select the gender in which you identify.-Response,Which duration range best aligns with your tenure at your company?-Response,Which of the following best describes your employment type?-Response,question_&_subquestion,Answer
0,5379192392,Infrastructure,NaN,Staff,Generation X (born between 1965-1980),Male,0-2 years,Full time Employee,Question 1-Response,NaN
1,2658722536,Finance,NaN,Staff,NaN,NaN,10+ years,Full time Employee,Question 1-Response,Answer 4
2,4044163394,Infrastructure,NaN,Department Lead,Generation X (born between 1965-1980),Male,3-5 years,Full time Employee,Question 1-Response,Answer 5
3,5535865599,Infrastructure,NaN,Manager,Millennial (born between 1981-2000),Non-Binary,5-10 years,Full time Employee,Question 1-Response,Answer 1
4,3356802928,Port Operations,NaN,Manager,Generation X (born between 1965-1980),Female,10+ years,Full time Employee,Question 1-Response,NaN


In [18]:
#Let's merge the questions dataframe with the melted dataframe, in order to know what the original question was

In [44]:
df_modified=pd.merge(left=df_melted,right=questions, how='left',left_on='question_&_subquestion', right_on='question_&_subquestion')
df_modified.tail()

,Respondent ID,Identify which division you work in.-Response,Identify which division you work in.-Other (please specify),Which of the following best describes your position level?-Response,Which generation are you apart of?-Response,Please select the gender in which you identify.-Response,Which duration range best aligns with your tenure at your company?-Response,Which of the following best describes your employment type?-Response,question_&_subquestion,Answer,Question
17023,7940065082,Infrastructure,NaN,Department Lead,Baby Boomer (born between 1946-1964),Male,10+ years,Full time Employee,Question 30-Response 3,Answer 8,Question 30
17024,5157705612,Finance,NaN,Staff,Millennial (born between 1981-2000),Female,5-10 years,Full time Employee,Question 30-Response 3,Answer 6,Question 30
17025,9920755555,Port Operations,NaN,Staff,Millennial (born between 1981-2000),Female,3-5 years,Full time Employee,Question 30-Response 3,NaN,Question 30
17026,6638341389,Infrastructure,NaN,Manager,Millennial (born between 1981-2000),Female,3-5 years,Full time Employee,Question 30-Response 3,NaN,Question 30
17027,8114622230,Information Technology,NaN,Staff,Prefer not to answer,Male,5-10 years,Full time Employee,Question 30-Response 3,NaN,Question 30


In [45]:
#From here, I think the work is done, because you can get the addtional information using programs like:
#Power Bi, Tableau or even with pivot tables in Excel. But, part of the task was to do this in python
df_modified.tail()

,Respondent ID,Identify which division you work in.-Response,Identify which division you work in.-Other (please specify),Which of the following best describes your position level?-Response,Which generation are you apart of?-Response,Please select the gender in which you identify.-Response,Which duration range best aligns with your tenure at your company?-Response,Which of the following best describes your employment type?-Response,question_&_subquestion,Answer,Question
17023,7940065082,Infrastructure,NaN,Department Lead,Baby Boomer (born between 1946-1964),Male,10+ years,Full time Employee,Question 30-Response 3,Answer 8,Question 30
17024,5157705612,Finance,NaN,Staff,Millennial (born between 1981-2000),Female,5-10 years,Full time Employee,Question 30-Response 3,Answer 6,Question 30
17025,9920755555,Port Operations,NaN,Staff,Millennial (born between 1981-2000),Female,3-5 years,Full time Employee,Question 30-Response 3,NaN,Question 30
17026,6638341389,Infrastructure,NaN,Manager,Millennial (born between 1981-2000),Female,3-5 years,Full time Employee,Question 30-Response 3,NaN,Question 30
17027,8114622230,Information Technology,NaN,Staff,Prefer not to answer,Male,5-10 years,Full time Employee,Question 30-Response 3,NaN,Question 30


In [26]:
#We have to create a new dataframe with none NaN Answer
Respondents=df_modified[df_modified['Answer'].notna()]
Respondents.tail()

,Respondent ID,Identify which division you work in.-Response,Identify which division you work in.-Other (please specify),Which of the following best describes your position level?-Response,Which generation are you apart of?-Response,Please select the gender in which you identify.-Response,Which duration range best aligns with your tenure at your company?-Response,Which of the following best describes your employment type?-Response,question_&_subquestion,Answer,Question
17017,9735550076,People,NaN,Staff,Generation X (born between 1965-1980),Male,5-10 years,Full time Employee,Question 30-Response 3,Answer 5,Question 30
17020,7325851635,Infrastructure,NaN,Manager,Millennial (born between 1981-2000),Prefer not to answer,3-5 years,Full time Employee,Question 30-Response 3,Answer 5,Question 30
17021,3370365802,Port Security & Emergency Operations,NaN,Staff,Generation X (born between 1965-1980),Female,0-2 years,Full time Employee,Question 30-Response 3,Answer 6,Question 30
17023,7940065082,Infrastructure,NaN,Department Lead,Baby Boomer (born between 1946-1964),Male,10+ years,Full time Employee,Question 30-Response 3,Answer 8,Question 30
17024,5157705612,Finance,NaN,Staff,Millennial (born between 1981-2000),Female,5-10 years,Full time Employee,Question 30-Response 3,Answer 6,Question 30


In [21]:
#Then we groupby Question, the nunique values, taking into account that we count the ID of respondents in every question
Num_Respondents=Respondents.groupby('Question')['Respondent ID'].nunique().reset_index() 
Num_Respondents.rename(columns={'Respondent ID':'Num_Respondents'}, inplace=True)
Num_Respondents.head()

,Question,Num_Respondents
0,Question 1,119
1,Question 10,198
2,Question 11,164
3,Question 12,114
4,Question 13,108


In [22]:
#We look for the number of equal answer per question and create a new df with the num_answers, I also changed the names of the column 'Respondent ID'
Num_answers=df_modified.groupby(['question_&_subquestion','Answer'])['Respondent ID'].nunique().reset_index()
Num_answers.rename(columns={'Respondent ID':'Num_answers'}, inplace=True)
Num_answers.head()

,question_&_subquestion,Answer,Num_answers
0,Question 1-Response,Answer 1,14
1,Question 1-Response,Answer 2,10
2,Question 1-Response,Answer 3,13
3,Question 1-Response,Answer 4,17
4,Question 1-Response,Answer 5,22


In [24]:
#We have to merge the Num_answer and the Num_respondents, and create a new df, 
df_output=df_modified.merge(Num_Respondents, how='left', on='Question').merge(Num_answers,how='left', on=['question_&_subquestion','Answer'])
df_output['Num_answers'].fillna(0,inplace=True) #Also we replace the NaN values with 0
df_output.tail()

,Respondent ID,Identify which division you work in.-Response,Identify which division you work in.-Other (please specify),Which of the following best describes your position level?-Response,Which generation are you apart of?-Response,Please select the gender in which you identify.-Response,Which duration range best aligns with your tenure at your company?-Response,Which of the following best describes your employment type?-Response,question_&_subquestion,Answer,Question,Num_Respondents,Num_answers
17023,7940065082,Infrastructure,NaN,Department Lead,Baby Boomer (born between 1946-1964),Male,10+ years,Full time Employee,Question 30-Response 3,Answer 8,Question 30,182,14.0
17024,5157705612,Finance,NaN,Staff,Millennial (born between 1981-2000),Female,5-10 years,Full time Employee,Question 30-Response 3,Answer 6,Question 30,182,20.0
17025,9920755555,Port Operations,NaN,Staff,Millennial (born between 1981-2000),Female,3-5 years,Full time Employee,Question 30-Response 3,NaN,Question 30,182,0.0
17026,6638341389,Infrastructure,NaN,Manager,Millennial (born between 1981-2000),Female,3-5 years,Full time Employee,Question 30-Response 3,NaN,Question 30,182,0.0
17027,8114622230,Information Technology,NaN,Staff,Prefer not to answer,Male,5-10 years,Full time Employee,Question 30-Response 3,NaN,Question 30,182,0.0


In [25]:
#Finaly we change the column'names with something more suitable
df_output.columns #To get the list of each column

#WE change the columns name with a dict and the method .rename
df_output.rename(columns={'Identify which division you work in.-Response':'Division',
       'Identify which division you work in.-Other (please specify)':'Division secondary',
       'Which of the following best describes your position level?-Response':'Position',
       'Which generation are you apart of?-Response':'Generation',
       'Please select the gender in which you identify.-Response':'Gender',
       'Which duration range best aligns with your tenure at your company?-Response':'Tenure',
       'Which of the following best describes your employment type?-Response':'Employment type'}, inplace=True)

In [ ]:
#The document is ready to be exported 
df_output.to_csv('/Final_Output.csv', index=False)
!cp /Final_Output.csv "drive/My Drive/" 